<a href="https://colab.research.google.com/github/aonekoda/ml_one_day/blob/main/04.Imbalanced_Class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 불균형한 클래스 처리하기

실전에서는 불균형한 클래스의 경우가 많다. 예를 들어 희귀함의 경우 샘플의 수가 매우 적을 수 밖에 없다. 이런 이유 때문에 불균형한 클래스를 다루는 일은 머신러닝에서 흔하다.  
가장 좋은 방법의 소수 클래스의 샘플을 더 많이 모으는 것이지만 이것이 불가능한 경우가 많기 때문에 다른 선택 사항을 고려해야 한다.  

In [3]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris

In [4]:
#  불균형한 클래스의 생성
iris = load_iris()
features = iris.data
target = iris.target

features = features[40:, :]
target = target[40:]

target = np.where((target ==0), 0, 1)
target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

### Class_Weights의 사용
머신러닝 모형은 class_weight옵션을 지원하는데 이를 통해 알고리즘이 불균형한 클래스를 조정할 수 있다.

In [5]:
#  명시적으로 불균등한 클래스에 가중치를 부여한다.
weights = {0: .9, 1:.1}

In [7]:
# 머신러닝 알고리즘에서 "class_weight"옵션을 제공한다.
LogisticRegression(class_weight=weights)

LogisticRegression(class_weight={0: 0.9, 1: 0.1})

In [8]:
# class_weight"옵션을 balanced로 설정하면 클래스 빈도에 반비려하여 자동으로 가중치를 만들어 준다.

LogisticRegression(class_weight="balanced")

LogisticRegression(class_weight='balanced')

### 다운샘플링(Down-sampling), 업샘플링(up-sampling)
* 다운샘플링 : 다수 클래스의 샘플 수를 줄인다.
* 업샘플링 : 소수 클래스의 샘플 수를 늘린다.

In [9]:
# 각 클래스의 샘플 인덱스 추출
i_class0 = np.where(target==0)[0]
i_class1 = np.where(target==1)[0]

In [10]:
# 각 클래스의 샘플 갯수
n_class0 = len(i_class0)
n_class1 = len(i_class1)

print(n_class0, n_class1)

10 100


### 다운샘플링

In [11]:
# 클래스0의 샘플만큼 클래스 1에서 중복을 허용하지 않고 랜덤 샘플
i_class1_downsampled = np.random.choice(i_class1, size=n_class0, replace=False)

In [12]:
# 클래스0와 클래스1을 합친다.

new_target = np.hstack((target[i_class0], target[i_class1_downsampled]))
new_features = np.vstack((features[i_class0, :], features[i_class1_downsampled, :]))

In [13]:
print(new_target)
print(new_features)

[0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1]
[[5.  3.5 1.3 0.3]
 [4.5 2.3 1.3 0.3]
 [4.4 3.2 1.3 0.2]
 [5.  3.5 1.6 0.6]
 [5.1 3.8 1.9 0.4]
 [4.8 3.  1.4 0.3]
 [5.1 3.8 1.6 0.2]
 [4.6 3.2 1.4 0.2]
 [5.3 3.7 1.5 0.2]
 [5.  3.3 1.4 0.2]
 [6.7 2.5 5.8 1.8]
 [5.5 2.4 3.7 1. ]
 [6.  2.2 4.  1. ]
 [5.  2.  3.5 1. ]
 [6.5 3.2 5.1 2. ]
 [6.  2.2 5.  1.5]
 [6.4 2.8 5.6 2.2]
 [6.7 3.3 5.7 2.1]
 [6.3 2.5 5.  1.9]
 [6.3 2.7 4.9 1.8]]


### 업샘플링

In [14]:
# 클래스1의 샘플만큼 클래스 0에서 중복을 허용하여 랜덤 샘플
i_class0_upsampled = np.random.choice(i_class0, size=n_class1, replace=True)

In [15]:
# 클래스0와 클래스1을 합친다.

new_target = np.hstack((target[i_class0_upsampled], target[i_class1]))
new_features = np.vstack((features[i_class0_upsampled, :], features[i_class1, :]))

In [ ]:
print(new_target)
print(new_features)

### 결론
* 머신러닝의 일부 모형은 class_weight옵션을 지원하는데 이를 통해 알고리즘이 불균형한 클래스를 조정할 수 있다. sklearn에서 제공하는 대부분의 분류 모형은 이 옵션을 지원한다. 도움말, 매뉴얼을 참조하자.
* 다운샘플링과 업샘플링을 통해 데이터의 크기를 임의로 조절한다. 어떤 것을 사용할지 여부는 문제에 따라 다르다. 일반적으로 둘 다 시도해 보고 더 나은 결과를 선택하도록 하자.